In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('wiki.csv')

# Display the first few rows of the dataset
print(data.head())


       id                                               text  \
0    2341  বংশী বাংলাদেশের একটি ক্ষুদ্রতম উপজাতি। এরা টাঙ...   
1  107282  আবহাওয়াবিদ্যা (বা আবহবিদ্যা) মানে এককথায় আবহ...   
2  107838  কোরিয়ার ওয়ার্কার্স পার্টি হল গণতান্ত্রিক গণপ...   
3    4542  মোহাম্মদ মোস্তফা কামাল (১৬ ডিসেম্বর ১৯৪৭ - এপ্...   
4    4626  মণিপুরী সংস্কৃতির উজ্জ্বলতম দিক হলো মণিপুরী নৃ...   

                         title                                         url  
0                         বংশী    https://bn.wikipedia.org/wiki?curid=2341  
1               আবহাওয়াবিদ্যা  https://bn.wikipedia.org/wiki?curid=107282  
2  কোরিয়ার ওয়ার্কার্স পার্টি  https://bn.wikipedia.org/wiki?curid=107838  
3   মোস্তফা কামাল (বীরশ্রেষ্ঠ)    https://bn.wikipedia.org/wiki?curid=4542  
4              মণিপুরী (নৃত্য)    https://bn.wikipedia.org/wiki?curid=4626  


In [2]:
 text_data = data['text']

text_data.head()

0    বংশী বাংলাদেশের একটি ক্ষুদ্রতম উপজাতি। এরা টাঙ...
1    আবহাওয়াবিদ্যা (বা আবহবিদ্যা) মানে এককথায় আবহ...
2    কোরিয়ার ওয়ার্কার্স পার্টি হল গণতান্ত্রিক গণপ...
3    মোহাম্মদ মোস্তফা কামাল (১৬ ডিসেম্বর ১৯৪৭ - এপ্...
4    মণিপুরী সংস্কৃতির উজ্জ্বলতম দিক হলো মণিপুরী নৃ...
Name: text, dtype: object

In [3]:
def preprocess_text(text):
    # Replace newlines and tabs with a single space
    text = text.replace('\n', ' ').replace('\t', ' ')
    # Optionally, add more preprocessing steps like removing special characters
    return text

# Apply preprocessing to each text entry
preprocessed_texts = text_data.apply(preprocess_text)


In [4]:
preprocessed_texts.head()

0    বংশী বাংলাদেশের একটি ক্ষুদ্রতম উপজাতি। এরা টাঙ...
1    আবহাওয়াবিদ্যা (বা আবহবিদ্যা) মানে এককথায় আবহ...
2    কোরিয়ার ওয়ার্কার্স পার্টি হল গণতান্ত্রিক গণপ...
3    মোহাম্মদ মোস্তফা কামাল (১৬ ডিসেম্বর ১৯৪৭ - এপ্...
4    মণিপুরী সংস্কৃতির উজ্জ্বলতম দিক হলো মণিপুরী নৃ...
Name: text, dtype: object

In [12]:
# Saving file
with open('bangla_wikipedia_corpus.txt', 'w', encoding='utf-8') as f:
    for text in preprocessed_texts:
        f.write(text + "\n")


In [6]:
import re

# Bengali Unicode range punctuation
bengali_unicode_range = re.compile(r'[\u0980-\u09FF]+')
basic_punctuation = re.compile(r'[\u0020-\u007E]+')

#  clean and normalize
def clean_text(text):
    # Replace Unicode symbols like [U+200E] with a space
    text = re.sub(r'\[U\+[0-9A-Fa-f]{4}\]', ' ', text)
    
    # Remove unwanted non-Bengali characters
    cleaned_text = ''.join([char if bengali_unicode_range.match(char) or basic_punctuation.match(char) else ' ' for char in text])
    
    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

# Function to ensure sentences end with correct Bengali punctuation
def correct_punctuation(text):
    # Replace full stops with Bengali dari if followed by a Bengali character
    text = re.sub(r'\.(?=\s*[\u0980-\u09FF])', '।', text)
    return text

# Initialize an empty string to hold the cleaned text
cleaned_corpus = ''

# process the files in chunks
with open('bangla_wikipedia_corpus.txt', 'r', encoding='utf-8') as infile:
    for line in infile:
        # Split into  chunks
        chunks = [line[i:i+500] for i in range(0, len(line), 500)]
        for chunk in chunks:
            cleaned_chunk = clean_text(chunk)
            corrected_chunk = correct_punctuation(cleaned_chunk)
            cleaned_corpus += corrected_chunk + ' '

# Save 
with open('cleaned_bangla_corpus.txt', 'w', encoding='utf-8') as outfile:
    outfile.write(cleaned_corpus)


In [10]:
import re

# Define the Bengali Unicode range and Bengali punctuation
bengali_unicode_range = re.compile(r'[\u0980-\u09FF]+')
bengali_punctuation = re.compile(r'[\u0980-\u09FF\u09F7-\u09FA\u200C-\u200D\u09E6-\u09EF]+|[\u0020-\u007E]+')

# Function to clean and normalize the text while retaining Bengali punctuation
def clean_text(text):
    # Replace Unicode symbols like [U+200E] with a space
    text = re.sub(r'\[U\+[0-9A-Fa-f]{4}\]', ' ', text)
    
    # Retain Bengali characters and punctuation, remove others
    cleaned_text = ''.join([char if bengali_unicode_range.match(char) or bengali_punctuation.match(char) else ' ' for char in text])
    
    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

# Function to split long lines
def split_long_lines(text, max_length):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# Initialize an empty list to hold the cleaned and split text
cleaned_lines = []

# Read and process the file in chunks
with open('bangla_wikipedia_corpus.txt', 'r', encoding='utf-8') as infile:
    for line in infile:
        cleaned_line = clean_text(line)
        split_lines = split_long_lines(cleaned_line, 10000)  # Split lines into chunks of 10,000 characters
        cleaned_lines.extend(split_lines)

# Save the cleaned and split text to a file
with open('final_cleaned_bangla_corpus.txt', 'w', encoding='utf-8') as outfile:
    for cleaned_line in cleaned_lines:
        outfile.write(cleaned_line + '\n')

print("Cleaning and splitting done. Saved to 'final_cleaned_bangla_corpus.txt'")


Cleaning and splitting done. Saved to 'final_cleaned_bangla_corpus.txt'


In [2]:
import sentencepiece as spm

In [11]:
import sentencepiece as spm

# Define paths and parameters
corpus_file = 'final_cleaned_bangla_corpus.txt'  # Path to your final cleaned corpus
model_prefix = 'bangla_bpe'  # Prefix for the output model files
vocab_size = 32000  # Set your desired vocabulary size

# Train the SentencePiece tokenizer
spm.SentencePieceTrainer.train(
    input=corpus_file,
    model_prefix=model_prefix,
    vocab_size=vocab_size,
    model_type='bpe',
    character_coverage=0.9995,  # Cover almost all characters in Bangla
    max_sentence_length=10000  # Maximum number of characters in a sentence
)

print(f"Tokenizer trained and saved as {model_prefix}.model and {model_prefix}.vocab")


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: final_cleaned_bangla_corpus.txt
  input_format: 
  model_prefix: bangla_bpe
  model_type: BPE
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 10000
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential

KeyboardInterrupt: 

bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1461 size=3300 all=236090 active=16129 piece=▁টুয়েন্টি
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=1459 min_freq=207
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1450 size=3320 all=236881 active=12591 piece=▁যাওয়া
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1441 size=3340 all=237790 active=13500 piece=▁নীতি
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1430 size=3360 all=239243 active=14953 piece=▁নিষ
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1424 size=3380 all=240206 active=15916 piece=▁তাহ
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1408 size=3400 all=241130 active=16840 piece=াগত
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=1408 min_freq=200
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1401 size=3420 all=242461 active=13323 piece=▁প্রত্যেক
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1385 size=3440 all=243524 active=14386 piece=াহ্ম
bpe_model_trainer.cc(268) 

In [13]:
import sentencepiece as spm

# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor(model_file='bangla_bpe.model')

# Example sentences
texts = [
    "এটি একটি উদাহরণ বাক্য।",
    "বাংলা একটি সুন্দর ভাষা।",
    "আমরা ভাষা মডেল তৈরি করছি।"
]

# Encode and print tokenized sentences
for text in texts:
    encoded_pieces = sp.encode(text, out_type=str)
    print(f"Original: {text}")
    print(f"Encoded pieces: {encoded_pieces}")
    print()

# Decode back to text
for text in texts:
    encoded_pieces = sp.encode(text, out_type=str)
    decoded_text = sp.decode(encoded_pieces)
    print(f"Encoded pieces: {encoded_pieces}")
    print(f"Decoded text: {decoded_text}")
    print()


Original: এটি একটি উদাহরণ বাক্য।
Encoded pieces: ['▁এটি', '▁একটি', '▁উদাহরণ', '▁বাক্য', '।']

Original: বাংলা একটি সুন্দর ভাষা।
Encoded pieces: ['▁বাংলা', '▁একটি', '▁সুন্দর', '▁ভাষা', '।']

Original: আমরা ভাষা মডেল তৈরি করছি।
Encoded pieces: ['▁আমরা', '▁ভাষা', '▁মডেল', '▁তৈরি', '▁করছি', '।']

Encoded pieces: ['▁এটি', '▁একটি', '▁উদাহরণ', '▁বাক্য', '।']
Decoded text: এটি একটি উদাহরণ বাক্য।

Encoded pieces: ['▁বাংলা', '▁একটি', '▁সুন্দর', '▁ভাষা', '।']
Decoded text: বাংলা একটি সুন্দর ভাষা।

Encoded pieces: ['▁আমরা', '▁ভাষা', '▁মডেল', '▁তৈরি', '▁করছি', '।']
Decoded text: আমরা ভাষা মডেল তৈরি করছি।



In [ ]:
import sentencepiece as spm

# Define paths and parameters
corpus_file = 'final_cleaned_bangla_corpus.txt'  
model_prefix = 'bangla_char'  
vocab_size = 32000 


spm.SentencePieceTrainer.train(
    input=corpus_file,
    model_prefix=model_prefix,
    vocab_size=vocab_size,
    model_type='char',  
    character_coverage=1.0,  
    max_sentence_length=10000,  
    input_sentence_size=1000000,  
    shuffle_input_sentence=True,  
    train_extremely_large_corpus=True  
)

print(f"Character-level tokenizer trained and saved as {model_prefix}.model and {model_prefix}.vocab")


In [ ]:
import sentencepiece as spm

# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor(model_file='bangla_char.model')

# Example function to tokenize text
def tokenize(text):
    return sp.encode(text, out_type=int)

# Example usage
text = "এটি একটি উদাহরণ বাক্য।"
token_ids = tokenize(text)
print(f"Token IDs: {token_ids}")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset


sp = spm.SentencePieceProcessor(model_file='bangla_char_bpe.model')


texts = [
    "এটি একটি উদাহরণ বাক্য।",
    "বাংলা একটি সুন্দর ভাষা।",
    "আমরা ভাষা মডেল তৈরি করছি।"
]


class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        token_ids = self.tokenizer.encode(text, out_type=int)
        return torch.tensor(token_ids)


dataset = CustomDataset(texts, sp)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


trainer.train()
